# 

# Custom Parsing
LUME-base objects can be instantiated using a single YAML file or a series of YAML files defining configuration options. The top-level file defines the base configuration options: 
- input_file
- initial_particles
- command
- command_mpi
- use_mpi
- mpi_run
- use_temp_dir
- workdir
- verbost
- timeout 

The `from_yaml` class method may be used to instantiate the `Base` subclass using the top-level configuration file. Developers may implement their own logic for parsing implementation-specific input files passed to the configuration. Below, we implement the abstract, static method `input_parser` for handling images files referenced in the yaml file pertinent to `MyModel`.

In [1]:
from random import random
from lume.base import CommandWrapper, tools
import os
import yaml
import numpy as np
import inspect


# Print code
from IPython.display import display, Markdown
def sdisplay(obj):
    spec = inspect.getsource(obj)
    display(Markdown(f"```python \n {spec} \n ```"))


`MyModel` implements static method and adds placeholders for other abstract methods:

In [2]:
from lume.tests.files.test_command_wrapper_subclass import MyModel
from lume.tests.files import LUME_CONFIG_YAML, INPUT_YAML 

sdisplay(MyModel)

```python 
 class MyModel(CommandWrapper):
    def __init__(self, *args, variables=None, input_image=None, **kwargs):
        super().__init__(*args, **kwargs)
        self._input_image = input_image
        self._variables = variables


    #implementation of abstract method
    @staticmethod
    def input_parser(path):
        config = {}

        if os.path.exists(tools.full_path(path)):
            yaml_file = tools.full_path(path)
            config = yaml.safe_load(open(yaml_file))


            # convention for this is that any input file paths are relative the input 
            # yaml directory
            if "input_image" in config:
                
                # Get the yaml file root
                root, _ = os.path.split(tools.full_path(path))
                input_image_path = os.path.join(root, config["input_image"])

                if os.path.exists(tools.full_path(input_image_path)):
                    with open(input_image_path, "rb") as f:
                        config["input_image"] = np.load(f)

                else:
                    print(f"unable to resolve input impage path {input_image_path}")

        else:
            print(f"unable to parse model input file at {path}")
                
        return config

    def archive(self):
        ...

    def configure(self):
        ...

    def load_archive(self):
        ...

    def load_output(self):
        ...

    def plot(self):
        ...

    def run(self):
        ...

    def write_input(self):
        ...
 
 ```

In this case, out configuration file looks like:

In [3]:
with open(LUME_CONFIG_YAML, "r") as stream:
    print(stream.read())

input_file: test_input_file.yml
use_temp_dir: false
use_mpi: false
timeout: 100



And the `input_file` looks like:

In [4]:
with open(INPUT_YAML, "r") as stream:
    print(stream.read())

input_image: test_input_image.npy

variables:
  variable_1:
    value: 1

  variable_2:
    value: 2



Instantiate model:

In [5]:
MyModel = MyModel.from_yaml(LUME_CONFIG_YAML)

print("Input image:")
print(MyModel._input_image)
print("Variables:")
print(MyModel._variables)

Input image:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Variables:
{'variable_1': {'value': 1}, 'variable_2': {'value': 2}}
